## **Image Classification Model - CNN**
# Summer School Session 6

This is the follow up code to the session 6, which describes the basics of CNNs, their properties, and their components.

Here we will be using PyTorch, so let us import the libraries.

In [ ]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from IPython.display import IFrame

Now, we will setup a GPU to be used (if available).

In [ ]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Let us define the relevant hyperparameters here, so that they can be accessed and changed easily.

In [ ]:
# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

For this example, we will be using the famous MNIST dataset, which is already available in PyTorch to be downloaded and used directly.<br>
Wikipedia page of the dataset: https://en.wikipedia.org/wiki/MNIST_database <br>
Here, we are downloading the test and train sets.

In [ ]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())


Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw




Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw




Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw




Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Here are some examples of images taken from the MNIST dataset. <br> They are handwritten digits, from 0-9.
![MNIST examples](https://machinelearningmastery.com/wp-content/uploads/2019/02/Plot-of-a-Subset-of-Images-from-the-MNIST-Dataset.png)

Now, let us utilise the PyTorch feature of dataloaders to load the data for use in the model.

In [ ]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

Next, we will define the CNN model we will be using for this task.<br>
Notice the use of convolutional, pooling, and fully connected (linear) layers in this model, which we had described in detail in the slides.<br>
Also pay attention to the usage of batchnorm, which, as explained in the slides, helps in training.

In [ ]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

We then initialize an object of the model defined.

In [ ]:
model = ConvNet(num_classes).to(device)


Next, let us define the loss function and optimizer to be used.

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Now, let us run the training loop for the model to generate predictions.

In [ ]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch [1/5], Step [100/600], Loss: 0.1060
Epoch [1/5], Step [200/600], Loss: 0.1683
Epoch [1/5], Step [300/600], Loss: 0.0962
Epoch [1/5], Step [400/600], Loss: 0.0546
Epoch [1/5], Step [500/600], Loss: 0.0337
Epoch [1/5], Step [600/600], Loss: 0.0363
Epoch [2/5], Step [100/600], Loss: 0.0062
Epoch [2/5], Step [200/600], Loss: 0.0642
Epoch [2/5], Step [300/600], Loss: 0.0323
Epoch [2/5], Step [400/600], Loss: 0.0146
Epoch [2/5], Step [500/600], Loss: 0.0324
Epoch [2/5], Step [600/600], Loss: 0.1064
Epoch [3/5], Step [100/600], Loss: 0.0653
Epoch [3/5], Step [200/600], Loss: 0.0163
Epoch [3/5], Step [300/600], Loss: 0.0190
Epoch [3/5], Step [400/600], Loss: 0.0190
Epoch [3/5], Step [500/600], Loss: 0.0190
Epoch [3/5], Step [600/600], Loss: 0.0645
Epoch [4/5], Step [100/600], Loss: 0.0045
Epoch [4/5], Step [200/600], Loss: 0.0818
Epoch [4/5], Step [300/600], Loss: 0.0392
Epoch [4/5], Step [400/600], Loss: 0.0378
Epoch [4/5], Step [500/600], Loss: 0.0337
Epoch [4/5], Step [600/600], Loss:

Finally, we will run the evaluation loop to test the model's predictions.

In [ ]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 99.08 %


And that's it! We've just gone through a very basic CNN in PyTorch, and seen how it can be used to classify the MNIST dataset.

Source for the code: https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/02-intermediate/convolutional_neural_network/main.py

#Take-home Assignment
Follow the directions in the image below for the assignment.

In [ ]:
###RUN THIS CELL TO SEE THE ASSIGNMENT QUESTION###
IFrame("https://drive.google.com/file/d/1nBzPpgRRz1ei0ZLvaArm_vs9amiYL2c1/preview", width=1080, height=720)